<h1>Aim 3</h1>
<h3>WIDS 2024 Challenge ++</h3>
<h3>BMI 212 - Team DMMTS</h3>

<h4>Manoj Maddali, MD</h4>

In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score, roc_curve, RocCurveDisplay, auc, mean_squared_error, accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn import preprocessing
import matplotlib.pyplot as plt

<h4> Load the data</h4>

In [3]:
# Load the CSV dataset
nonimpute_raw_df = pd.read_csv('../Data/train_test_added_climate_data.csv')
impute_raw_df = pd.read_csv('../Data/train_test_added_climate_data_imputed.csv')

# Rename feature columns for better readability
columns_dict = {'bmi': 'patient_bmi',
                   'region': 'patient_region',
                   'division': 'patient_division',
                   'side': 'patient_tumor_side',
                   'quadrant': 'patient_tumor_quadrant',
                   'metastatic_organ': 'patient_metastatic_organ',
                   'cleaned_metastatic_first_treatment': 'patient_metastatic_first_treatment',
                   'cleaned_metastatic_first_treatment_type': 'patient_metastatic_first_treatment_type',
                   'population': 'population_size',
                   'density': 'population_density',
                   'age_median': 'population_age_median',
                   'female': 'population_female_perc',
                   'married': 'population_married_perc',
                   'divorced': 'population_divorced_perc',
                   'never_married': 'population_never_married_perc',
                   'widowed': 'population_widowed_perc',
                   'family_size': 'population_family_size',
                   'family_dual_income': 'population_family_dual_income_perc',
                   'income_individual_median': 'population_income_individual_median',
                   'income_household_median': 'population_income_household_median',
                   'home_ownership': 'population_home_ownership_perc',
                   'home_value': 'population_home_value',
                   'rent_median': 'population_rent_median',
                   'rent_burden': 'population_rent_burden_perc',
                   'education_less_highschool': 'population_education_less_highschool_perc',
                   'education_highschool': 'population_education_highschool_perc',
                   'education_some_college': 'population_education_some_college_perc',
                   'education_bachelors': 'population_education_bachelors_perc',
                   'education_graduate': 'population_education_graduate_perc',
                   'education_college_or_above': 'population_education_college_or_above_perc',
                   'education_stem_degree': 'population_education_stem_degree_perc',
                   'unemployment_rate': 'population_unemployment_rate',
                   'self_employed': 'population_self_employed_perc',
                   'farmer': 'population_farmer_perc',
                   'race_white': 'population_race_white_perc',
                   'race_black': 'population_race_black_perc',
                   'race_asian': 'population_race_asian_perc',
                   'race_native': 'population_race_native_american_perc',
                   'race_pacific': 'population_race_pacific_islander_perc',
                   'race_other': 'population_race_other_perc',
                   'race_multiple': 'population_race_multiple_perc',
                   'hispanic': 'population_hispanic_perc',
                   'disabled': 'population_disabled_perc',
                   'poverty': 'population_poverty_perc',
                   'limited_english': 'population_limited_english_perc',
                   'commute_time': 'population_commute_time',
                   'health_uninsured': 'population_health_uninsured_perc',
                   'veteran': 'population_veteran_perc',
                   'climate_ozone': 'annual_ozone_conc',
                   'climate_pm25': 'annual_fine_particulate_matter_conc',
                   'climate_n02': 'annual_nitrogen_dioxide_conc',
                   'Ozone': 'annual_ozone_conc',
                   'PM25': 'annual_fine_particulate_matter_conc',
                   'N02': 'annual_nitrogen_dioxide_conc'}

nonimpute_raw_df.rename(columns=columns_dict, inplace=True)
impute_raw_df.rename(columns=columns_dict, inplace=True)

<h4>Select the features to use</h4>

In [4]:
features = ['patient_race', 'payer_type', 'patient_state', 'patient_age', 'patient_gender', 'patient_bmi',
            'patient_region', 'patient_division', 'patient_tumor_side', 'patient_tumor_quadrant',
            'patient_metastatic_organ', 'patient_metastatic_first_treatment', 'patient_metastatic_first_treatment_type',
            'population_size', 'population_density', 'population_age_median', 'population_female_perc',
            'population_married_perc', 'population_divorced_perc', 'population_never_married_perc',
            'population_widowed_perc', 'population_family_size', 'population_family_dual_income_perc',
            'population_income_individual_median', 'population_income_household_median', 'population_home_ownership_perc',
            'population_home_value', 'population_rent_median', 'population_rent_burden_perc',
            'population_education_less_highschool_perc', 'population_education_highschool_perc',
            'population_education_some_college_perc', 'population_education_bachelors_perc',
            'population_education_graduate_perc', 'population_education_college_or_above_perc',
            'population_education_stem_degree_perc', 'population_unemployment_rate', 'population_self_employed_perc',
            'population_farmer_perc', 'population_race_white_perc', 'population_race_black_perc',
            'population_race_asian_perc', 'population_race_native_american_perc', 'population_race_pacific_islander_perc',
            'population_race_other_perc', 'population_race_multiple_perc', 'population_hispanic_perc',
            'population_disabled_perc', 'population_poverty_perc', 'population_limited_english_perc',
            'population_commute_time', 'population_health_uninsured_perc', 'population_veteran_perc', 'annual_nitrogen_dioxide_conc',
            'annual_fine_particulate_matter_conc', 'annual_ozone_conc']

# Select only rows where allocated_set is train
nonimpute_train_df = nonimpute_raw_df[nonimpute_raw_df['allocated_set'] == 'train']
impute_train_df = impute_raw_df[impute_raw_df['allocated_set'] == 'train']

# Select the features to use
nonimpute_df = nonimpute_train_df[features]
impute_df = impute_train_df[features]

# Extract labels for time to treatment 
labels_df = nonimpute_train_df[['treatment_pd']]

<h4>Define categorical variables</h4>

In [6]:
# Convert object features to categorical
for col in nonimpute_df.select_dtypes(include='object').columns:
    nonimpute_df[col] = nonimpute_df[col].astype('category')
    impute_df[col] = impute_df[col].astype('category')

# List of categorical features
categorical_features = list(nonimpute_df.select_dtypes(include='category').columns)
print(categorical_features)

['patient_race', 'payer_type', 'patient_state', 'patient_gender', 'patient_region', 'patient_division', 'patient_tumor_side', 'patient_tumor_quadrant', 'patient_metastatic_organ', 'patient_metastatic_first_treatment', 'patient_metastatic_first_treatment_type']


In [7]:
# Temporarily convert categorical features to distinct numerical codes, keeping missing/NaN values
temp_nonimpute_df = nonimpute_df.copy()
temp_impute_df = impute_df.copy()

for cat_feat in categorical_features:
    temp_nonimpute_df[cat_feat] = temp_nonimpute_df[cat_feat].cat.codes
    temp_impute_df[cat_feat] = temp_impute_df[cat_feat].cat.codes

    temp_nonimpute_df.loc[temp_nonimpute_df[cat_feat] == -1] = np.NaN
    temp_impute_df.loc[temp_impute_df[cat_feat] == -1] = np.NaN

In [8]:
from sklearn.feature_selection import VarianceThreshold

# Remove low-var features from temp df (will drop removed cols from original df)
low_var_filter_nonimpute = VarianceThreshold(threshold=(.9 * (1 - .9)))
filtered_features_nonimpute = low_var_filter_nonimpute.fit_transform(temp_nonimpute_df)
filtered_feature_names_nonimpute = low_var_filter_nonimpute.get_feature_names_out(input_features=features)
print("Non-imputed Low-variance features: ", set(features) - set(filtered_feature_names_nonimpute))

low_var_filter_impute = VarianceThreshold(threshold=(.9 * (1 - .9)))
filtered_features_impute = low_var_filter_impute.fit_transform(temp_impute_df)
filtered_feature_names_impute = low_var_filter_impute.get_feature_names_out(input_features=features)
print("Imputed Low-variance features: ", set(features) - set(filtered_feature_names_impute))

nonimpute_df_filtered = nonimpute_df[filtered_feature_names_nonimpute].copy()
impute_df_filtered = impute_df[filtered_feature_names_impute].copy()

Non-imputed Low-variance features:  {'patient_gender', 'population_family_size', 'patient_metastatic_first_treatment_type'}
Imputed Low-variance features:  {'patient_gender', 'population_family_size', 'patient_metastatic_first_treatment_type'}


# Categorize label to 30/60/90 day treatment

In [9]:
# Rename dfs of features and target
X_nonimpute = nonimpute_df_filtered
X_impute = impute_df_filtered
y = labels_df

y_30 = y['treatment_pd'].apply(lambda x: 1 if x <= 30 else 0)

y_60 = y['treatment_pd'].apply(lambda x: 1 if x <= 60 else 0)

y_90 = y['treatment_pd'].apply(lambda x: 1 if x <= 90 else 0)

# Split into train/test 80/20

In [10]:
# Split the data into train/test split (80/20) - preserve class counts between splits
X_train_nonimpute_30, X_test_nonimpute_30, y_train_30, y_test_30 = train_test_split(X_nonimpute, y_30, test_size=0.2, random_state=123, stratify=y_30)
X_train_impute_30, X_test_impute_30, y_train_30, y_test_30 = train_test_split(X_impute, y_30, test_size=0.2, random_state=123, stratify=y_30)
print(y_train_30.value_counts())
print(y_test_30.value_counts())

X_train_nonimpute_60, X_test_nonimpute_60, y_train_60, y_test_60 = train_test_split(X_nonimpute, y_60, test_size=0.2, random_state=123, stratify=y_60)
X_train_impute_60, X_test_impute_60, y_train_60, y_test_60 = train_test_split(X_impute, y_60, test_size=0.2, random_state=123, stratify=y_60)
print(y_train_60.value_counts())
print(y_test_60.value_counts())

X_train_nonimpute_90, X_test_nonimpute_90, y_train_90, y_test_90 = train_test_split(X_nonimpute, y_90, test_size=0.2, random_state=123, stratify=y_90)
X_train_impute_90, X_test_impute_90, y_train_90, y_test_90 = train_test_split(X_impute, y_90, test_size=0.2, random_state=123, stratify=y_90)
print(y_train_90.value_counts())
print(y_test_90.value_counts())

0    17714
1     4425
Name: treatment_pd, dtype: int64
0    4429
1    1106
Name: treatment_pd, dtype: int64
0    12685
1     9454
Name: treatment_pd, dtype: int64
0    3171
1    2364
Name: treatment_pd, dtype: int64
1    13549
0     8590
Name: treatment_pd, dtype: int64
1    3387
0    2148
Name: treatment_pd, dtype: int64


# Imputed - LightGBM

<h4>Imputed 30-day classification</h4>

In [11]:
# Create the LightGBM dataset
train_data_30 = lgb.Dataset(X_train_impute_30, label=y_train_30, feature_name='auto', categorical_feature=categorical_features)
test_data_30 = lgb.Dataset(X_test_impute_30, label=y_test_30, feature_name='auto', categorical_feature=categorical_features)

# Define the hyperparameters for a classification model
params = {
    'objective': ['binary'],
    'metric': ['auc'], 
    'is_unbalance': [True],
    'boosting_type': ['dart'], 
    'n_estimators': [50, 100],
    'num_leaves': [31, 63],
    'learning_rate': [0.05, 0.1],
    'colsample_bytree': [0.9],
    'subsample': [0.9],
    'subsample_freq': [10],
    'verbose': [-1],
}

# Train the model
model_impute_30 = lgb.LGBMClassifier()

# Create the grid search
grid_impute_30 = GridSearchCV(model_impute_30, params, cv=10, scoring='roc_auc')

# Fit the model to the data
grid_impute_30.fit(X_train_impute_30, y_train_30)

GridSearchCV(cv=10, estimator=LGBMClassifier(),
             param_grid={'boosting_type': ['dart'], 'colsample_bytree': [0.9],
                         'is_unbalance': [True], 'learning_rate': [0.05, 0.1],
                         'metric': ['auc'], 'n_estimators': [50, 100],
                         'num_leaves': [31, 63], 'objective': ['binary'],
                         'subsample': [0.9], 'subsample_freq': [10],
                         'verbose': [-1]},
             scoring='roc_auc')

In [12]:
# Print best parameters
print('Best parameters:', grid_impute_30.best_params_)

# Print best in-sample score
print('Insample AUC:', grid_impute_30.best_score_)

# Test the model
y_test_pred_impute_lgb_30 = grid_impute_30.predict_proba(X_test_impute_30)[:,1]

# Calculate AUC
fpr_30, tpr_30, thresholds = roc_curve(y_test_30, y_test_pred_impute_lgb_30)

# Roc AUC
roc_auc_30 = auc(fpr_30, tpr_30)

# Print AUC
print('AUC:', roc_auc_30)

Best parameters: {'boosting_type': 'dart', 'colsample_bytree': 0.9, 'is_unbalance': True, 'learning_rate': 0.05, 'metric': 'auc', 'n_estimators': 50, 'num_leaves': 31, 'objective': 'binary', 'subsample': 0.9, 'subsample_freq': 10, 'verbose': -1}
Insample AUC: 0.7001098101655292
AUC: 0.7138134243439896


# Fairness - Equalized odds

In [15]:
from fairlearn.metrics import equalized_odds_ratio, equalized_odds_difference, MetricFrame, true_positive_rate, false_positive_rate

equalized_odds_metrics = {
    'true_positive_rate': true_positive_rate,
    'false_positive_rate': false_positive_rate}

## By Race
equalized_odds_race = MetricFrame(metrics=equalized_odds_metrics,
                                  y_true=y_test_30,
                                  y_pred=(y_test_pred_impute_lgb_30 > 0.5).astype(int),
                                  sensitive_features=X_test_impute_30['patient_race'])

print('Equalized odds by race:', equalized_odds_race.by_group)

print('Equalized odds ratio by race:', equalized_odds_ratio(y_true=y_test_30,
                                                            y_pred=(y_test_pred_impute_lgb_30 > 0.5).astype(int),
                                                            sensitive_features = X_test_impute_30['patient_race']))

print('Equalized odds difference by race:', equalized_odds_difference(y_true=y_test_30,
                                                                     y_pred=(y_test_pred_impute_lgb_30 > 0.5).astype(int),
                                                                     sensitive_features = X_test_impute_30['patient_race']))

Equalized odds by race:               true_positive_rate  false_positive_rate
patient_race                                         
Asian                   0.533333             0.207207
Black                   0.579545             0.295905
Hispanic                0.523364             0.285714
Other                   0.602410             0.300787
White                   0.630719             0.303439
Equalized odds ratio by race: 0.6828622022309253
Equalized odds difference by race: 0.10735446826705763


# Before/After Mitigation with Reductions

In [42]:
from fairlearn.reductions import ExponentiatedGradient, EqualizedOdds

mitigator = ExponentiatedGradient(grid_impute_30.best_estimator_, constraints=EqualizedOdds())

mitigator.fit(X_train_impute_30, y_train_30, sensitive_features=X_train_impute_30['patient_race'])

y_test_pred_impute_lgb_30_reduction = mitigator.predict(X_test_impute_30)

In [43]:
from fairlearn.metrics import equalized_odds_ratio, equalized_odds_difference, MetricFrame, true_positive_rate, false_positive_rate

equalized_odds_metrics = {
    'true_positive_rate': true_positive_rate,
    'false_positive_rate': false_positive_rate}

## By Race
equalized_odds_race = MetricFrame(metrics=equalized_odds_metrics,
                                  y_true=y_test_30,
                                  y_pred=(y_test_pred_impute_lgb_30_reduction > 0.5).astype(int),
                                  sensitive_features=X_test_impute_30['patient_race'])

print('Equalized odds by race:', equalized_odds_race.by_group)

print('Equalized odds ratio by race:', equalized_odds_ratio(y_true=y_test_30,
                                                            y_pred=(y_test_pred_impute_lgb_30_reduction > 0.5).astype(int),
                                                            sensitive_features = X_test_impute_30['patient_race']))

print('Equalized odds difference by race:', equalized_odds_difference(y_true=y_test_30,
                                                                     y_pred=(y_test_pred_impute_lgb_30_reduction > 0.5).astype(int),
                                                                     sensitive_features = X_test_impute_30['patient_race']))

Equalized odds by race:               true_positive_rate  false_positive_rate
patient_race                                         
Asian                   0.511111             0.220721
Black                   0.448864             0.244386
Hispanic                0.420561             0.264479
Other                   0.493976             0.266142
White                   0.486928             0.224205
Equalized odds ratio by race: 0.8228362454286876
Equalized odds difference by race: 0.09055036344755968
